In [53]:
import os
import pandas as pd 
import numpy as np
import math
from sklearn.model_selection import KFold
import time
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
print('Libraries Imported')

Libraries Imported


In [2]:
def quaternion_to_euler(x, y, z, w):

    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.degrees(math.atan2(t3, t4))

    return X, Y, Z

In [3]:
print(os.listdir("./"))

['X_train.csv.zip', 'X_test.csv.zip', 'y_train.csv', 'X_test.csv', 'train_transform', 'Untitled.ipynb', 'sample_submission.csv', 'lgb_submit.csv', 'test_transform', 'lightgbm.ipynb', '.ipynb_checkpoints', 'X_train.csv']


In [4]:
trainX = pd.read_csv('X_train.csv')
trainY = pd.read_csv('y_train.csv')
testX = pd.read_csv('X_test.csv')

In [24]:
trainX_new=pd.read_csv('train_transform',sep='\t')
testX_new=pd.read_csv('test_transform',sep='\t')

In [25]:
print(trainX_new.shape)
print(trainX_new.iloc[0,3:7])
trainX_new.drop(columns='orientation_W',axis=1, inplace=True)
print(trainX_new.iloc[0,])
testX_new.drop(columns='orientation_W',axis=1, inplace=True)

(487680, 14)
measurement_number          0
orientation_X         162.908
orientation_Y        -1.41335
orientation_Z          80.023
Name: 0, dtype: object
Unnamed: 0                        0
row_id                          0_0
series_id                         0
measurement_number                0
orientation_X               162.908
orientation_Y              -1.41335
orientation_Z                80.023
angular_velocity_X          0.10765
angular_velocity_Y         0.017561
angular_velocity_Z       0.00076741
linear_acceleration_X      -0.74857
linear_acceleration_Y         2.103
linear_acceleration_Z       -9.7532
Name: 0, dtype: object


In [32]:
trainX_new['new1']=trainX.iloc[:,3]
trainX_new['new2']=trainX.iloc[:,4]
trainX_new['new3']=trainX.iloc[:,5]
trainX_new['new4']=trainX.iloc[:,6]
testX_new['new1']=testX.iloc[:,3]
testX_new['new2']=testX.iloc[:,4]
testX_new['new3']=testX.iloc[:,5]
testX_new['new4']=testX.iloc[:,6]

In [40]:
print(testX_new.iloc[0,])

Unnamed: 0                       0
row_id                         0_0
series_id                        0
measurement_number               0
orientation_X              162.991
orientation_Y            -0.816677
orientation_Z              177.135
angular_velocity_X      -0.0065237
angular_velocity_Y      -0.0010714
angular_velocity_Z        -0.02739
linear_acceleration_X      0.10043
linear_acceleration_Y       4.2061
linear_acceleration_Z      -5.5439
new1                     -0.025773
new2                      -0.98864
new3                      -0.14801
new4                       0.00335
Name: 0, dtype: object


In [14]:
param = {'num_leaves': 10,
         'min_data_in_leaf': 10, 
         'objective':'multiclass',
         'num_class': 9,
         'max_depth': 2,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'multi_error',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1}
max_iter = 5


In [34]:
target = trainY.iloc[:,2]
print(type(target))
#target=targeto
targetc=pd.factorize(target)[0]
print(type(targetc))
print(targetc.min())
print(targetc.max())
target=targetc
#target = pd.DataFrame(label_binarize(targetc, classes=[1,2,3,4,5,6,7,8,9]))
print(target.shape)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
0
8
(3810,)


In [38]:
print(trainX_new.shape)
print(len(target))
#for i in range(0,len(target)):
print(type(trainX_new.iloc[0,4:17]))
col_names =  range(0,128*13)
trainX_new1  = pd.DataFrame(columns = col_names)
print(trainX_new1.shape)
for i in range(0,len(target)):
    c=[]
    for j in range(0,128):
        c=c+list(trainX_new.iloc[i*128+j,4:17])
    trainX_new1.loc[i]=c
print(trainX_new1.shape)    
#print(len(c))

(487680, 17)
3810
<class 'pandas.core.series.Series'>
(0, 1664)
(3810, 1664)


In [41]:
print(testX_new.shape)
col_names =  range(0,128*13)
testX_new1  = pd.DataFrame(columns = col_names)
print(testX_new1.shape)
for i in range(0,int(testX_new.shape[0]/128)):
    c=[]
    for j in range(0,128):
        c=c+list(testX_new.iloc[i*128+j,4:17])
    testX_new1.loc[i]=c
print(testX_new1.shape)    

(488448, 17)
(0, 1664)
(3816, 1664)


In [11]:
trainX_new1.dtypes

0       float64
1       float64
2       float64
3       float64
4       float64
5       float64
6       float64
7       float64
8       float64
9       float64
10      float64
11      float64
12      float64
13      float64
14      float64
15      float64
16      float64
17      float64
18      float64
19      float64
20      float64
21      float64
22      float64
23      float64
24      float64
25      float64
26      float64
27      float64
28      float64
29      float64
         ...   
1122    float64
1123    float64
1124    float64
1125    float64
1126    float64
1127    float64
1128    float64
1129    float64
1130    float64
1131    float64
1132    float64
1133    float64
1134    float64
1135    float64
1136    float64
1137    float64
1138    float64
1139    float64
1140    float64
1141    float64
1142    float64
1143    float64
1144    float64
1145    float64
1146    float64
1147    float64
1148    float64
1149    float64
1150    float64
1151    float64
Length: 1152, dtype: obj

In [101]:
clf=RandomForestClassifier(n_estimators=500,criterion='entropy',random_state=222,max_depth=10,min_samples_leaf=4,min_samples_split=5)
#print(cross_val_score(clf,trainX_new1,target,cv=5))


In [102]:
clf.fit(trainX_new1,target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=222, verbose=0, warm_start=False)

In [96]:
predictions=clf.predict(trainX_new1[3001:3810])

In [97]:
print(target[3001:3010])
print(predictions[0:10])
print(len(target[3001:3810] == predictions))

[0 1 0 4 0 8 0 1 4]
[0 1 0 4 0 8 0 1 4 8]
809


In [103]:
predictions=clf.predict(testX_new1)

In [104]:
targetx = trainY.iloc[:,2]
cate=targetx.unique()
predictions_new=cate[predictions]
sub_df = pd.DataFrame({"series_id": list(range(3816))})
sub_df["surface"] = predictions_new
sub_df[:10]
sub_df.to_csv('rf_submit.csv',index=False)

In [100]:
print(predictions[:5])
print(predictions_new[:5])

[5 1 3 6 2]
['hard_tiles_large_space' 'concrete' 'tiled' 'carpet' 'soft_tiles']


In [99]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros((len(target),9))
#categorical_columns = [c for c in categorical_columns if c not in ['MachineIdentifier']]
#print(trainX_new1.iloc[1,])
features = trainX_new1.columns
predictions = np.zeros((len(testX_new1),9))
start = time.time()
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(trainX_new1.values, target.values)):
    print("fold n°{}".format(fold_))
    print(len(trn_idx))
    print(len(val_idx))
    print(len(features))
    trn_data = lgb.Dataset(trainX_new1.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           #categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(trainX_new1.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           #categorical_feature = categorical_columns
                          )

    #print(trn_data.shape)
    #print(val_data.shape)
    num_round = 100
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    print(trainX_new1.iloc[val_idx][features].shape)
    print(trainX_new1.iloc[trn_idx][features].shape)
    #tmp = clf.predict(trainX_new1.iloc[val_idx][features], num_iteration=clf.best_iteration)
    #print(tmp.shape)
    oof[val_idx] = clf.predict(trainX_new1.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    # we perform predictions by chunks
    initial_idx = 0
    chunk_size = 1000000
    current_pred = np.zeros((len(testX_new1),9))
    while initial_idx < testX_new1.shape[0]:
        final_idx = min(initial_idx + chunk_size, testX_new1.shape[0])
        idx = range(initial_idx, final_idx)
        current_pred[idx] = clf.predict(testX_new1.iloc[idx][features], num_iteration=clf.best_iteration)
        initial_idx = final_idx
    predictions += current_pred / min(folds.n_splits, max_iter)
   
    print("time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
    #score[fold_] = metrics.roc_auc_score(target.iloc[val_idx], oof[val_idx])
    tmp1=np.argmax(target.iloc[val_idx],axis=1)
    tmp2=np.argmax(oof[val_idx],axis=1)
    score[fold_] = metrics.accuracy_score(tmp1,tmp2)
    if fold_ == max_iter - 1: break
        
if (folds.n_splits == max_iter):
    #print("CV score: {:<8.5f}".format(metrics.roc_auc_score(target, oof)))
    tmp1=np.argmax(target,axis=1)
    tmp2=np.argmax(oof,axis=1)
    print("CV score: {:<8.5f}".format(metrics.accuracy_score(tmp1,tmp2)))
else:
     print("CV score: {:<8.5f}".format(sum(score) / max_iter))

fold n°0
3048
762
1152
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_error: 0.451115	valid_1's multi_error: 0.479003
Did not meet early stopping. Best iteration is:
[100]	training's multi_error: 0.451115	valid_1's multi_error: 0.479003
(762, 1152)
(3048, 1152)
time elapsed: 0.0031s
fold n°1
3048
762
1152
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_error: 0.461942	valid_1's multi_error: 0.514436
Did not meet early stopping. Best iteration is:
[100]	training's multi_error: 0.461942	valid_1's multi_error: 0.514436
(762, 1152)
(3048, 1152)
time elapsed: 0.0061s
fold n°2
3048
762
1152
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_error: 0.47605	valid_1's multi_error: 0.518373
Did not meet early stopping. Best iteration is:
[100]	training's multi_error: 0.47605	valid_1's multi_error: 0.518373
(762, 1152)
(3048, 1152)
time elapsed: 0.0091s
fold n°3
3048
762
1152
Training u

In [75]:
print(score)
print(max_iter)

[0.028871391076115485, 0.02230971128608924, 0.03543307086614173, 0.030183727034120734, 0.03674540682414698]
5


In [90]:
targetx = trainY.iloc[:,2]
cate=targetx.unique()
predictions_new=np.argmax(predictions,axis=1)
print(predictions[0,])
print(predictions_new[0])
print(cate[predictions_new[0]])
predictions_new1=cate[predictions_new]
print(predictions_new1[0])

[6.70141775e-02 6.32916360e-02 4.07728323e-02 6.13804365e-02
 1.21029419e-01 6.43578701e-02 1.33642516e-01 1.27046866e-05
 3.17130938e-01]
8
wood
wood


In [41]:
print(list(range(10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [91]:
sub_df = pd.DataFrame({"series_id": list(range(3816))})


In [92]:
print(len(predictions_new1))
print(sub_df.shape)
sub_df["surface"] = predictions_new1
sub_df[:10]

3816
(3816, 1)


,series_id,surface
0,0,wood
1,1,concrete
2,2,tiled
3,3,wood
4,4,tiled
5,5,concrete
6,6,concrete
7,7,concrete
8,8,concrete
9,9,soft_pvc


In [93]:
sub_df.to_csv('lgb_submit.csv',index=False)